In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые
from sklearn.model_selection import train_test_split
# импортируем классы BaseEstimator и TransformerMixin, 
# позволяющие написать собственные классы
from sklearn.base import BaseEstimator, TransformerMixin
# импортируем классы и функции, необходимые для корректной работы
# пользовательских классов RareFrouper и FreqImputer
from pandas import Series, DataFrame
from collections import Counter, defaultdict
# импортируем класс SimpleImputer, позволяющий
# выполнить импутацию пропусков
from sklearn.impute import SimpleImputer
# импортируем класс StandardScaler, позволяющий
# выполнить стандартизацию
from sklearn.preprocessing import StandardScaler
# импортируем класс OneHotEncoder, позволяющий
# выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс ColumnTransformer, позволяющий выполнять
# преобразования для отдельных типов столбцов
from sklearn.compose import ColumnTransformer
# импортируем класс FunctionTransformer, позволяющий
# задавать пользовательские функции
from sklearn.preprocessing import FunctionTransformer
# импортируем класс Pipeline, позволяющий создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс GridSearchCV, позволяющий 
# выполнить поиск по сетке
from sklearn.model_selection import GridSearchCV
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression
# импортируем функцию roc_auc_score() для вычисления AUC-ROC
from sklearn.metrics import roc_auc_score
# импортируем класс StratifiedKFold для выполнения
# стратифицированной перекрестной проверки
from sklearn.model_selection import StratifiedKFold

In [2]:
# увеличиваем количество выводимых столбцов
pd.set_option('display.max_columns', 60)
# загружаем набор данных
data = pd.read_csv('Data/Credit_OTP.csv', encoding='cp1251', sep=';')
# выводим первые 5 наблюдений
data.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000,6,8650,0,1,220,18.0,0,0,1,1,1,6,2,1,1580,1580,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650,6,4000,0,1,137,97.0,1,0,1,1,1,6,1,1,4020,4020,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126,12,4000,0,1,251,84.0,0,0,1,2,1,11,0,0,0,0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,"8491,82",6,5000,0,0,36,168.0,1,1,1,1,1,6,3,1,"1589,923333",1590,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990,12,4000,0,1,83,101.0,1,0,1,2,1,16,2,1,"1152,15",2230,NaN


In [3]:
# выводим информацию о количестве непропущенных 
# наблюдений в переменных и типах переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null int64
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null int64
SOCSTATUS_PENS_FL             15223 non-null int64
GENDER                        15223 non-null int64
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  13856 non-null object
GEN_TITLE                     13856 non-null object
ORG_TP_STATE                  13856 non-null object
ORG_TP_FCAPITAL               13858 non-null object
JOB_DIR                       13856 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null object
REG_ADDRESS_PRO

In [4]:
# переменные, неверно записанные как количественные,
# преобразуем в тип object
object_columns = data.columns[data.columns.str.contains('FL|GENDER|OWN_AUTO|AGREEMENT')]
data[object_columns] = data[object_columns].astype('object')

# в указанных переменных заменяем запятую на точку в качестве
# десятичного разделителя и преобразуем в тип float
for col in ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 
               'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']:
    data[col] = data[col].str.replace(',', '.').astype('float')
    
# вновь смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null object
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null object
SOCSTATUS_PENS_FL             15223 non-null object
GENDER                        15223 non-null object
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  13856 non-null object
GEN_TITLE                     13856 non-null object
ORG_TP_STATE                  13856 non-null object
ORG_TP_FCAPITAL               13858 non-null object
JOB_DIR                       13856 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null float64
REG_ADDRES

In [5]:
# создаем список из переменных GEN_ INDUSTRY, GEN_TITLE, 
# ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR
work_cols = data.columns[data.columns.str.contains('GEN_I|GEN_T|JOB|ORG_')].tolist()

# заменяем пропуски в переменных GEN_ INDUSTRY, GEN_TITLE, 
# ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR  на "Не указано",
# если в интересующей нас переменной есть пропуск 
# И при этом переменная SOCSTATUS_PENS_FL имеет значение 1
for col in work_cols:
    data[col] = np.where((data[col].isnull()) & (data['SOCSTATUS_PENS_FL'] == 1),
                         'Не указано', data[col])
    
# заменяем пропуски в переменных GEN_ INDUSTRY, GEN_TITLE, 
# ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR  модами,
# если в интересующей нас переменной есть пропуск 
# И при этом переменная SOCSTATUS_PENS_FL имеет значение 0
for col in work_cols:
    data[col] = np.where((data[col].isnull()) & (data['SOCSTATUS_PENS_FL'] == 0),
                         data[col].value_counts().index[0], data[col])

# заменяем пропуски в переменной REGION_NM модой
data['REGION_NM'] = np.where(data['REGION_NM'].isnull(), 
                             data['REGION_NM'].value_counts().index[0], 
                             data['REGION_NM'])

# пропуски в переменной TP_PROVINCE заменим значением
# переменной FACT_ADDRESS_PROVINCE
data['TP_PROVINCE'] = np.where(data['TP_PROVINCE'].isnull(), 
                               data['FACT_ADDRESS_PROVINCE'], 
                               data['TP_PROVINCE'])
    
# заменяем пропуски в переменной 
# PREVIOUS_CARD_NUM_UTILIZED нулями
data['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(data['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, 
                                              data['PREVIOUS_CARD_NUM_UTILIZED'])

In [6]:
# выводим статистики по категориальным переменным,
# смотрим unique - количество уникальных значений,
# выявляем бесполезные переменные - переменные, у
# которых уникальных значений столько, сколько
# наблюдений (AGREEMENT_RK), и переменные с одним
# уникальным значением (DL_DOCUMENT_FL)
data.select_dtypes(include='object').describe()

,AGREEMENT_RK,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL
count,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223
unique,15223,2,2,2,7,5,32,13,6,3,11,5,81,81,80,72,11,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2
top,64585727,1,0,1,Среднее специальное,Состою в браке,Торговля,Специалист,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,Краснодарский край,Краснодарский край,Краснодарский край,Краснодарский край,ЮЖНЫЙ,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1
freq,1,13847,13176,9964,6518,9416,2386,7011,6524,13689,11453,7077,674,674,674,790,2180,12765,14480,13493,12758,13782,10507,13453,14097,11940,15074,14930,14558,15223,8961,9311,14389,13855


In [7]:
# удаляем идентификационную переменную AGREEMENT_RK,
# потому что у нее количество уникальных значений
# равно количеству наблюдений, переменную DL_DOCUMENT_FL, 
# потому что у нее одно уникальное значение
data.drop(['AGREEMENT_RK', 'DL_DOCUMENT_FL'], axis=1, inplace=True)

In [8]:
# заново выделим подмножество категориальных переменных
# и пробежим по нему, выводя частоты категорий
# по каждой категориальной переменной, чтобы
# выявить редкие категории
for col in data.select_dtypes(include='object').columns:
    print(data[col].value_counts(dropna=False))

1    13847
0     1376
Name: SOCSTATUS_WORK_FL, dtype: int64
0    13176
1     2047
Name: SOCSTATUS_PENS_FL, dtype: int64
1    9964
0    5259
Name: GENDER, dtype: int64
Среднее специальное               6518
Среднее                           4679
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: EDUCATION, dtype: int64
Состою в браке        9416
Не состоял в браке    3622
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: MARITAL_STATUS, dtype: int64
Торговля                                     2386
Другие сферы                                 1709
Не указано                                   1366
Металлургия/Промышленность/Машиностроение    1356
Государственная служба                       1286
Здравоохранение                              1177
Образование                                   998
Транспорт 

In [9]:
# заменяем неверную категорию "ПОВОЛЖСКИЙ" на категорию "ПРИВОЛЖСКИЙ"
data.loc[data['REGION_NM'] == 'ПОВОЛЖСКИЙ', 'REGION_NM'] = 'ПРИВОЛЖСКИЙ'

In [10]:
# укрупняем категории переменной EDUCATION
ed_lst = ['Ученая степень', 'Два и более высших образования']
data.loc[data['EDUCATION'].isin(ed_lst), 'EDUCATION'] = 'Высшее'

In [11]:
# укрупняем категории переменной OWN_AUTO
data.loc[data['OWN_AUTO'] == '2', 'OWN_AUTO'] = '1'

In [12]:
# создаем словарь со строковыми значениями для замены
dct = {'Агинский Бурятский АО': 'Забайкальский край', 
       'Читинская область': 'Забайкальский край',
       'Усть-Ордынский Бурятский АО': 'Иркутская область',
       'Эвенкийский АО': 'Красноярский край'}

# заменяем категории в переменных REG_ADDRESS_PROVINCE, POSTAL_ADDRESS_PROVINCE, 
# FACT_ADDRESS_PROVINCE, TP_PROVINCE
for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 
            'FACT_ADDRESS_PROVINCE', 'TP_PROVINCE']:
    data[col].replace(dct, inplace=True)

In [13]:
# записываем редкие категории переменных REG_ADDRESS_PROVINCE, 
# POSTAL_ADDRESS_PROVINCE, FACT_ADDRESS_PROVINCE в одну 
# отдельную категорию
for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE']:
    data[col] = np.where((data[col] == 'Хакасия')
                       | (data[col] == 'Ямало-Ненецкий АО')
                       | (data[col] == 'Магаданская область')
                       | (data[col] == 'Калмыкия')
                       | (data[col] == 'Дагестан')
                       | (data[col] == 'Чечня'), 
                       'ДРУГОЕ', data[col])

# записываем редкие категории переменной TP_PROVINCE в одну отдельную категорию
data['TP_PROVINCE'] = np.where((data['TP_PROVINCE'] == 'Сахалинская область')
                               | (data['TP_PROVINCE'] == 'Еврейская АО')
                               | (data['TP_PROVINCE'] == 'Магаданская область')
                               | (data['TP_PROVINCE'] == 'Дагестан')
                               | (data['TP_PROVINCE'] == 'Кабардино-Балкария'), 
                               'ДРУГОЕ', data['TP_PROVINCE'])

# записываем некоторые категории переменной JOB_DIR 
# в отдельную категорию
jobdir_lst = ['Реклама и маркетинг', 'Кадровая служба и секретариат',
              'Пр-техн. обесп. и телеком.', 'Юридическая служба']
data.loc[data['JOB_DIR'].isin(jobdir_lst), 'JOB_DIR'] = 'Другое'

In [14]:
# создаем переменную FACT_TP_FL, которая принимает значение 1, если 
# область фактического пребывания клиента и область торговой точки, 
# где клиент брал последний кредит, совпадают, или 0
# в противном случае
data['FACT_TP_FL'] = np.where(data['FACT_ADDRESS_PROVINCE'] == data['TP_PROVINCE'], 
                              1, 0).astype('object')

In [15]:
# отключаем экспоненциальное представление
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# выводим статистики по количественным переменным
data.describe()

,TARGET,AGE,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,OWN_AUTO,CREDIT,TERM,FST_PAYMENT,FACT_LIVING_TERM,WORK_TIME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
count,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,13855.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000
mean,0.119,40.406,1.099,0.645,13853.836,0.116,14667.959,8.101,3398.563,3039.340,292.212,1.388,0.752,7.876,0.195,0.130,239.457,243.022,0.019
std,0.324,11.601,0.995,0.812,9015.468,0.321,12147.873,4.094,5158.109,262455.437,24364.832,0.794,0.988,7.956,0.685,0.356,827.700,836.100,0.138
min,0.000,21.000,0.000,0.000,24.000,0.000,2000.000,3.000,0.000,-26.000,1.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,30.000,0.000,0.000,8000.000,0.000,6500.000,6.000,1000.000,41.000,24.000,1.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000
50%,0.000,39.000,1.000,0.000,12000.000,0.000,11550.000,6.000,2000.000,108.000,48.000,1.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
75%,0.000,50.000,2.000,1.000,17000.000,0.000,19170.000,10.000,4000.000,204.000,110.000,2.000,1.000,8.000,0.000,0.000,0.000,0.000,0.000
max,1.000,67.000,10.000,7.000,250000.000,2.000,119700.000,36.000,140000.000,28101997.000,2867959.000,11.000,11.000,114.000,13.000,8.000,28012.200,28012.200,2.000


##### Обращаем внимание на:
- отрицательное минимальное значение `FACT_LIVING_TERM`;
- на нулевые минимальные значения переменных `CHILD_TOTAL`, `DEPENDANTS`, `OWN_AUTO`, `FST_PAYMENT`, `LOAN_NUM_CLOSED`, `LOAN_DLQ_NUM`, `LOAN_MAX_DLQ`, `LOAN_AVG_DLQ_AMT`, `LOAN_MAX_DLQ_AMT`, `PREVIOUS_CARD_NUM_UTILIZED`, при конструировании новых признаков на базе таких переменных нужно быть особо внимательным, т.к. при делении на ноль могут появятся бесконечные значения (infinite values);
- отмечаем аномально большие максимальные значения переменных `FACT_LIVING_TERM` и `WORK_TIME`, такие значения могут снизить качество регрессионной модели

In [16]:
# значения переменной FACT_LIVING_TERM берем по модулю, 
# чтобы избавиться от отрицательных значений
data['FACT_LIVING_TERM'] = data['FACT_LIVING_TERM'].abs()

In [17]:
a = 0.001
# создаем переменную PAYMENT - сумму ежемесячного взноса
# по кредиту, разделив сумму кредита (CREDIT)
# на срок кредита (TERM)
data['PAYMENT'] = (data['CREDIT'] + a) / (data['TERM'] + a)

#создаем переменную SHARE_INCOME - долю дохода от суммы кредита
data['SHARE_INCOME'] = (data['PERSONAL_INCOME'] + a) / (data['CREDIT'] + a)

# вычитаем количество погашенных ссуд (LOAN_NUM_CLOSED)
# из общего количество ссуд (LOAN_NUM_TOTAL)
data['LOAN_NUM_OPEN'] = data['LOAN_NUM_TOTAL'] - data['LOAN_NUM_CLOSED']

# создаем переменную PAYM_TO_LOAN – отношение количества 
# платежей (LOAN_NUM_PAYM) к общему количеству ссуд (LOAN_NUM_TOTAL)
data['PAYM_TO_LOAN'] = (data['LOAN_NUM_PAYM'] + a) / (data['LOAN_NUM_TOTAL'] + a)

# создаем переменную DLQ_TO_PAYM – отношение количества просрочек 
# (LOAN_DLQ_NUM) к общему количеству платежей (LOAN_NUM_PAYM)
data['DLQ_TO_PAYM'] = (data['LOAN_DLQ_NUM'] + a) / (data['LOAN_NUM_PAYM'] + a)

# создаем переменную FST_SHARE - отношение суммы первого платежа по кредиту
# к объединенной сумме первого платежа (FST_PAYMENT) и кредита (CREDIT) 
data['FST_SHARE'] = (data['FST_PAYMENT'] + a) / (data['FST_PAYMENT'] + data['CREDIT'])

In [18]:
# пишем функцию для укрупнения категорий
# переменной REGION_NM в конвейере
def collapse_categories(X):
    dct = {'ЮЖНЫЙ': 'cat1', 
           'ЦЕНТРАЛЬНЫЙ 1': 'cat1', 
           'ЦЕНТРАЛЬНЫЙ 2': 'cat1',
           'ЗАПАДНО-СИБИРСКИЙ': 'cat2', 
           'ВОСТОЧНО-СИБИРСКИЙ': 'cat2', 
           'УРАЛЬСКИЙ': 'cat2',
           'ДАЛЬНЕВОСТОЧНЫЙ': 'cat2', 
           'СЕВЕРО-ЗАПАДНЫЙ': 'cat2', 
           'ЦЕНТРАЛЬНЫЙ ОФИС': 'cat2',
           'ПРИВОЛЖСКИЙ': 'cat3'}
    X.replace(dct, inplace=True)
    return X

In [19]:
# на основе переменной PERSONAL_INCOME создаем временную переменную 
# PERSONAL_INCOME_CAT, у которой метки категорий будут 
# повторять метки категорий FAMILY_INCOME
bins = [-np.inf, 5000, 10000, 20000, 50000, np.inf]
lab = ['до 5000 руб.', 'от 5000 до 10000 руб.', 'от 10000 до 20000 руб.', 
       'от 20000 до 50000 руб.', 'свыше 50000 руб.']
data['PERSONAL_INCOME_CAT'] = pd.cut(data['PERSONAL_INCOME'], bins, labels=lab)

In [20]:
# выводим метки переменной PERSONAL_INCOME_CAT
data['PERSONAL_INCOME_CAT'].unique()

[до 5000 руб., от 10000 до 20000 руб., от 5000 до 10000 руб., от 20000 до 50000 руб., свыше 50000 руб.]
Categories (5, object): [до 5000 руб. < от 5000 до 10000 руб. < от 10000 до 20000 руб. < от 20000 до 50000 руб. < свыше 50000 руб.]

In [21]:
# создаем переменную PERSONAL_FAMILY_INCOME, которая принимает значение 1, если 
# категория переменной PERSONAL_INCOME_CAT совпадает с категорией 
# переменной FAMILY_INCOME, или 0 в противном случае, затем
# преобразовываем в тип object
data['PERSONAL_FAMILY_INCOME'] = np.where(data['PERSONAL_INCOME_CAT'] == data['FAMILY_INCOME'], 
                                           1, 0).astype('object')

# удаляем переменную PERSONAL_INCOME_CAT
data.drop('PERSONAL_INCOME_CAT', axis=1, inplace=True)

In [22]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
train, test, y_train, y_test = train_test_split(data.drop('TARGET', axis=1), 
                                                data['TARGET'], 
                                                test_size=.3, 
                                                stratify=data['TARGET'], 
                                                random_state=100)

In [23]:
# наблюдения, в которых количество лет проживания 
# по месту фактического пребывания превышает
# возраст, записываем как пропуски 
train['FACT_LIVING_TERM'] = np.where(train['FACT_LIVING_TERM'] / 12 > train['AGE'],
                                     np.NaN, train['FACT_LIVING_TERM'])
test['FACT_LIVING_TERM'] = np.where(test['FACT_LIVING_TERM'] / 12 > test['AGE'], 
                                    np.NaN, test['FACT_LIVING_TERM'])

In [24]:
# наблюдения, в которых разница между возрастом и временем работы в годах 
# меньше 16 (например, у 30-летнего время работы в годах составляет 20 лет,
# получается, он работает с 10 лет), записываем как пропуски 
train['WORK_TIME'] = np.where((train['AGE'] - train['WORK_TIME'] / 12) < 16, 
                              np.NaN, train['WORK_TIME'])
test['WORK_TIME'] = np.where((test['AGE'] - test['WORK_TIME'] / 12) < 16, 
                             np.NaN, test['WORK_TIME'])

In [25]:
# создаем список категориальных переменных
cat_columns = train.dtypes[train.dtypes == 'object'].index.tolist()
# создаем список количественных переменных
num_columns = train.dtypes[train.dtypes != 'object'].index.tolist()
# создаем одноэлементные списки для переменных 
# REGION_NM, GEN_INDUSTRY, GEN_TITLE
regionnm_regroup = ['REGION_NM']
genindustry_delrare = ['GEN_INDUSTRY']
gentitle_delrare = ['GEN_TITLE']

In [26]:
# удаляем из списка категориальных переменных
# переменные REGION_NM, GEN_INDUSTRY, GEN_TITLE
cat_columns = list(set(cat_columns).difference(set(
    regionnm_regroup + genindustry_delrare + gentitle_delrare)))
# смотрим итоговый список категориальных переменных
cat_columns

['GPF_DOCUMENT_FL',
 'FACT_TP_FL',
 'LAND_PRESENCE_FL',
 'JOB_DIR',
 'SOCSTATUS_PENS_FL',
 'COT_PRESENCE_FL',
 'PERSONAL_FAMILY_INCOME',
 'REG_FACT_POST_TP_FL',
 'TP_PROVINCE',
 'MARITAL_STATUS',
 'REG_ADDRESS_PROVINCE',
 'HS_PRESENCE_FL',
 'GAR_PRESENCE_FL',
 'EDUCATION',
 'ORG_TP_STATE',
 'FL_PRESENCE_FL',
 'AUTO_RUS_FL',
 'FAMILY_INCOME',
 'GENDER',
 'ORG_TP_FCAPITAL',
 'REG_POST_FL',
 'FACT_ADDRESS_PROVINCE',
 'REG_FACT_FL',
 'FACT_PHONE_FL',
 'REG_FACT_POST_FL',
 'POSTAL_ADDRESS_PROVINCE',
 'SOCSTATUS_WORK_FL',
 'GEN_PHONE_FL',
 'REG_PHONE_FL',
 'FACT_POST_FL']

In [27]:
# создаем список количественных переменных, для которых
# применяем логарифмическое преобразование
num_columns_log = ['WORK_TIME', 'PERSONAL_INCOME', 'PAYMENT', 'CREDIT', 'LOAN_AVG_DLQ_AMT']
# создаем одноэлементные списки для переменных 
# FACT_LIVING_TERM и FST_PAYMENT
fctlivingterm_corrlog0_5 = ['FACT_LIVING_TERM']
fstpayment_corrlog0_1 = ['FST_PAYMENT']
# объединим предыдущие списки в один
comb_log_lst = num_columns_log + fctlivingterm_corrlog0_5 + fstpayment_corrlog0_1
# удаляем из списка количественных переменных переменные WORK_TIME, 
# PERSONAL_INCOME, PAYMENT, CREDIT, LOAN_AVG_DLQ_AMT, 
# FACT_LIVING_TERM, FST_PAYMENT
num_columns = list(set(num_columns).difference(set(comb_log_lst)))
# смотрим итоговый список количественных переменных
num_columns

['PREVIOUS_CARD_NUM_UTILIZED',
 'LOAN_NUM_CLOSED',
 'LOAN_NUM_PAYM',
 'AGE',
 'DLQ_TO_PAYM',
 'TERM',
 'LOAN_DLQ_NUM',
 'DEPENDANTS',
 'OWN_AUTO',
 'CHILD_TOTAL',
 'LOAN_MAX_DLQ_AMT',
 'SHARE_INCOME',
 'LOAN_MAX_DLQ',
 'LOAN_NUM_TOTAL',
 'PAYM_TO_LOAN',
 'FST_SHARE',
 'LOAN_NUM_OPEN']

In [28]:
# смотрим исходные частоты категорий 
# переменной GEN_TITLE
train['GEN_TITLE'].value_counts(dropna=False)

Специалист                        4886
Рабочий                           2175
Не указано                         967
Служащий                           623
Руководитель среднего звена        484
Работник сферы услуг               410
Высококвалифиц. специалист         382
Руководитель высшего звена         300
Индивидуальный предприниматель     152
Другое                             108
Руководитель низшего звена          98
Военнослужащий по контракту         62
Партнер                              9
Name: GEN_TITLE, dtype: int64

In [29]:
# создаем собственный класс RareGrouper, укрупняющий редкие категории
class RareGrouper(BaseEstimator, TransformerMixin): 
    """
        Параметры:
        - threshold: минимально допустимая абсолютная частота, при которой замены не происходит
    """
    
    def __init__(self, threshold=20):
        
        # Все очень просто, когда значения представлены целыми числами или строками. 
        # Однако если они представлены в форме списков и словарей, значение (пустой 
        # список или словарь) нужно инициализировать при первом использовании ключа. 
        # defaultdict (дословно: словарь со значением по умолчанию) автоматизирует и 
        # упрощает этот процесс. Он похож на обычный словарь, за исключением одной 
        # особенности - при попытке обратиться к ключу, которого в нем нет, 
        # он сперва добавляет для него значение, используя функцию без аргументов, 
        # которая предоставляется при его создании (в нашем случае речь пойдет о функции 
        # list()).
        self.d = defaultdict(list)
        self.threshold = threshold
        
    def fit(self, X, y=None):
        # для каждой переменной вычисляем абсолютные частоты категорий (calc_col)
        # и получаем словарь вида
        # defaultdict(list,
        #             {'var1': ['категория выше порога', 'категория выше порога'],
        #              'var2': ['категория выше порога', 'категория выше порога']})
        for col in X.columns:
            abs_freq = X[col].value_counts()
            self.d[col] = abs_freq[abs_freq >= self.threshold].index.tolist()
        return self

    def transform(self, X):
        # создаем копию датафрейма
        X = X.copy()
        # для каждой переменной категории (строковые значения), которых нет 
        # в соответствующем списке словаря (их частоты ниже порога), 
        # относим к категории Other
        for col in X.columns:
            X[col] = np.where(X[col].isin(self.d[col]), X[col], 'Other')
        return X

In [30]:
# теперь проверим работу класса RareGrouper для одной переменной,
# категории с количеством менее 70 наблюдений запишем в
# отдельную категорию Other
train_copy = train.copy()
raregrouper = RareGrouper(threshold=70)
raregrouper.fit(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'] = raregrouper.transform(train_copy[['GEN_TITLE']])
# смотрим частоты категорий 
# переменной GEN_TITLE
train_copy['GEN_TITLE'].value_counts(dropna=False)

Специалист                        4886
Рабочий                           2175
Не указано                         967
Служащий                           623
Руководитель среднего звена        484
Работник сферы услуг               410
Высококвалифиц. специалист         382
Руководитель высшего звена         300
Индивидуальный предприниматель     152
Другое                             108
Руководитель низшего звена          98
Other                               71
Name: GEN_TITLE, dtype: int64

In [31]:
# смотрим исходные частоты категорий переменных
# REG_ADDRESS_PROVINCE, POSTAL_ADDRESS_PROVINCE
lst = ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE']
for col in lst:
    print(train_copy[col].value_counts())

Кемеровская область     481
Краснодарский край      450
Алтайский край          310
Забайкальский край      304
Приморский край         281
                       ... 
Марийская республика     55
Горный Алтай             52
Еврейская АО             46
Москва                   46
Сахалинская область      40
Name: REG_ADDRESS_PROVINCE, Length: 74, dtype: int64
Кемеровская область     481
Краснодарский край      449
Алтайский край          309
Забайкальский край      303
Приморский край         280
                       ... 
Камчатская область       59
Марийская республика     53
Горный Алтай             51
Еврейская АО             48
Сахалинская область      40
Name: POSTAL_ADDRESS_PROVINCE, Length: 74, dtype: int64


In [32]:
# теперь проверим работу класса RareGrouper для списка переменных,
# категории с количеством менее 200 наблюдений запишем в
# отдельную категорию Other
raregroup = RareGrouper(threshold=200)
raregroup.fit(train_copy[lst])
train_copy[lst] = raregroup.transform(train_copy[lst])
# смотрим частоты категорий переменных REG_ADDRESS_PROVINCE, 
# POSTAL_ADDRESS_PROVINCE
for col in lst:
    print(train_copy[col].value_counts())

Other                    7005
Кемеровская область       481
Краснодарский край        450
Алтайский край            310
Забайкальский край        304
Приморский край           281
Ростовская область        274
Свердловская область      239
Саратовская область       239
Воронежская область       218
Омская область            216
Волгоградская область     215
Вологодская область       212
Татарстан                 212
Name: REG_ADDRESS_PROVINCE, dtype: int64
Other                    7014
Кемеровская область       481
Краснодарский край        449
Алтайский край            309
Забайкальский край        303
Приморский край           280
Ростовская область        272
Свердловская область      238
Саратовская область       236
Воронежская область       219
Волгоградская область     215
Омская область            215
Вологодская область       213
Татарстан                 212
Name: POSTAL_ADDRESS_PROVINCE, dtype: int64


In [33]:
# создаем собственный класс RareGrouper2, укрупняющий редкие категории
class RareGrouper2(TransformerMixin, BaseEstimator):
    """
    Параметры:
    - min_freq: минимально допустимая абсолютная частота, при которой замены не происходит
    - max_features: максимально возможное количество категорий. Если None, то без ограничения
    - include_na: обрабатывать NaN как категорию или игнорировать
    - features: признаки, которые надо обрабатывать:
        - если на вход подается array, то индексы
        - если DataFrame, то список колонок
        - можно подать список колонок для array, но тогда методу .fit() передать fmap
    - ffil_str: чем заменить строковые значения
    - ffil_num: чем заменить числовые значения
    - return_df: должен ли метод .transform() возвращать DataFrame
    """
    def __init__(self,  min_freq=20, max_features=None, include_na=True, features=None, 
                 fill_str ='other', fill_num =-1, return_df=False):
        
        self.features = features
        self.max_features = max_features
        self.min_freq = min_freq
        self.include_na = include_na
        self.fill_str = fill_str
        self.fill_num = fill_num
        self.return_df = return_df
             
    def fit(self, X, y = None, **fit_params):
        # проверяем, является ли наш массив двумерным
        assert len(X.shape) == 2, 'Array must be 2-dim'
        
        self.fmap = None
        if 'fmap' in fit_params:
            self.fmap = fit_params['fmap']

        # создаем датафрейм, столбцы определяются fmap    
        X = DataFrame(X, columns = self.fmap)
        # либо берем все имеющиеся признаки, либо те, что
        # указаны в features
        if self.features is None:
            features = X.columns
        else:
            features = self.features
        
        # создаем пустой словарь
        self.feats_dict = {}
        
        for i in features:
            # создаем словарь, в котором ключами будут названия категорий, 
            # а значениями - абсолютные частоты категорий
            cnt = Counter(X[i])
            # если задано include_na=True, возвращаем множество с названиями max_features категорий
            # словаря cnt (метод most_common возвращает max_features наиболее часто встречающихся категорий 
            # в порядке убывания встречаемости, если max_features=None, возвращаются все категории)
            # при условии, что категория имеет частоту, которая равна или выше min_freq
            if self.include_na:
                keeplist = set([x for x, f in cnt.most_common(self.max_features) if f >= self.min_freq])
            # если задано include_na=False, возвращаем множество с названиями max_features категорий
            # словаря cnt при условии, что категория имеет частоту, которая равна или выше min_freq, и не является nan  
            else:
                keeplist = set([x for x, f in cnt.most_common(self.max_features) if f >= self.min_freq \
                                and x is not np.nan])
            
            # ключом словаря feats_dict будет название переменной, значением словаря feats_dict будет 
            # двухэлементный кортеж, первый элемент - множество с названиями 
            # категорий переменной, второй элемент - значение, определяемое 
            # параметром fill_num (если переменная хранит числа) или
            # параметром fill_str (если переменная хранит строковые значения)
            if np.issubdtype(X[i].dtype, np.number):
                self.feats_dict[i] = (keeplist, self.fill_num)
            else:
                self.feats_dict[i] = (keeplist, self.fill_str)
        return self
    
    def transform(self, X, y = None):
        
        # проверяем, является ли наш массив двумерным
        assert len(X.shape) == 2, 'Array must be 2-dim'
        # проверяем, был ли вызван метод fit перед
        # вызовом метода transform
        assert 'feats_dict' in self.__dict__, 'Model should be fitted first'
        
        # создаем датафрейм, столбцы определяются fmap
        out = DataFrame(X, columns = self.fmap)
        
        # в словаре для каждого ключа - переменной разбиваем 
        # значение - двухэлементный кортеж на два элемента l и v,
        # l - множество названий категорий, v - значение, которое
        # выше было определено значением параметра fill_num/fill_str,
        # категории, находящиеся ВНЕ множества l, заменяем v
        for i in self.feats_dict:
            l, v = self.feats_dict[i]
            out.loc[~out[i].isin(l), i] = v
        
        # если задано return_df=True, возвращаем 
        # данные в виде датафрейма
        if self.return_df:
            return out
        #если задано return_df=False, возвращаем массив NumPy
        else:
            return out.values

### Если тяжело разобраться, что именно делает класс, разложите его на более простые компоненты

In [34]:
# задаем значения параметров fill_str и fill_num
fill_str = 'Other'
fill_num = -1
# создаем пустой словарь
feats_dict = {}

for i in ['GEN_INDUSTRY', 'GEN_TITLE']:
    # создаем словарь, в котором ключами будут названия категорий, 
    # а значениями - абсолютные частоты категорий
    cnt = Counter(train[i])
    # создаем множество с названиями max_features категорий словаря cnt
    # (метод most_common возвращает max_features наиболее часто встречающихся категорий 
    # в порядке убывания встречаемости, если max_features=None, возвращаются все категории)
    # при условии, что категория имеет частоту, которая равна или выше min_freq
    keeplist = set([x for x, f in cnt.most_common(n=None) if f >= 20])
    
    # ключом словаря feats_dict будет название переменной, значением словаря feats_dict будет 
    # двухэлементный кортеж, первый элемент - множество с названиями 
    # категорий переменной, второй элемент - значение, определяемое 
    # параметром fill_num (если переменная хранит числа) или
    # параметром fill_str (если переменная хранит строковые значения)
    if np.issubdtype(data[i].dtype, np.number):
        feats_dict[i] = (keeplist, fill_num)
    else:
        feats_dict[i] = (keeplist, fill_str)

In [35]:
# давайте взглянем на частоты категорий GEN_TITLE
train['GEN_TITLE'].value_counts(dropna=False)

Специалист                        4886
Рабочий                           2175
Не указано                         967
Служащий                           623
Руководитель среднего звена        484
Работник сферы услуг               410
Высококвалифиц. специалист         382
Руководитель высшего звена         300
Индивидуальный предприниматель     152
Другое                             108
Руководитель низшего звена          98
Военнослужащий по контракту         62
Партнер                              9
Name: GEN_TITLE, dtype: int64

In [36]:
# посмотрим на исходный словарь cnt (по последней итерации, 
# т.е. по переменной GEN_TITLE)
cnt

Counter({'Специалист': 4886,
         'Служащий': 623,
         'Рабочий': 2175,
         'Высококвалифиц. специалист': 382,
         'Не указано': 967,
         'Руководитель среднего звена': 484,
         'Работник сферы услуг': 410,
         'Военнослужащий по контракту': 62,
         'Руководитель высшего звена': 300,
         'Руководитель низшего звена': 98,
         'Индивидуальный предприниматель': 152,
         'Другое': 108,
         'Партнер': 9})

In [37]:
# посмотрим на множество категорий keeplist, отвечающее условиям
# (по последней итерации, т.е. по переменной GEN_TITLE)
keeplist

{'Военнослужащий по контракту',
 'Высококвалифиц. специалист',
 'Другое',
 'Индивидуальный предприниматель',
 'Не указано',
 'Работник сферы услуг',
 'Рабочий',
 'Руководитель высшего звена',
 'Руководитель низшего звена',
 'Руководитель среднего звена',
 'Служащий',
 'Специалист'}

In [38]:
# смотрим итоговый словарь
feats_dict

{'GEN_INDUSTRY': ({'Банк/Финансы',
   'Государственная служба',
   'Другие сферы',
   'Здравоохранение',
   'Информационные технологии',
   'Информационные услуги',
   'Коммунальное хоз-во/Дорожные службы',
   'Металлургия/Промышленность/Машиностроение',
   'Наука',
   'Не указано',
   'Нефтегазовая промышленность',
   'Образование',
   'Развлечения/Искусство',
   'Ресторанный бизнес/Общественное питание',
   'СМИ/Реклама/PR-агенства',
   'Салоны красоты и здоровья',
   'Сборочные производства',
   'Сельское хозяйство',
   'Строительство',
   'Торговля',
   'Транспорт',
   'Химия/Парфюмерия/Фармацевтика',
   'ЧОП/Детективная д-ть',
   'Энергетика',
   'Юридические услуги/нотариальные услуги'},
  'Other'),
 'GEN_TITLE': ({'Военнослужащий по контракту',
   'Высококвалифиц. специалист',
   'Другое',
   'Индивидуальный предприниматель',
   'Не указано',
   'Работник сферы услуг',
   'Рабочий',
   'Руководитель высшего звена',
   'Руководитель низшего звена',
   'Руководитель среднего звена

In [39]:
# сформируем датафрейм из двух переменных
out = DataFrame(data, columns = ['GEN_INDUSTRY', 'GEN_TITLE'])

In [40]:
# в словаре для каждого ключа - переменной разбиваем 
# значение - двухэлементный кортеж на два элемента l и v,
# l - множество названий категорий, v - значение, которое
# выше было определено значением параметра fill_num/fill_str,
# категории, находящиеся ВНЕ множества l, заменяем v
for i in feats_dict:
    l, v = feats_dict[i]
    out.loc[~out[i].isin(l), i] = v

In [41]:
# взглянем на множество l (по последней итерации, 
# т.е. по переменной GEN_TITLE)
l

{'Военнослужащий по контракту',
 'Высококвалифиц. специалист',
 'Другое',
 'Индивидуальный предприниматель',
 'Не указано',
 'Работник сферы услуг',
 'Рабочий',
 'Руководитель высшего звена',
 'Руководитель низшего звена',
 'Руководитель среднего звена',
 'Служащий',
 'Специалист'}

In [42]:
# взглянем на значение v (по последней итерации, 
# т.е. по переменной GEN_TITLE)
v

'Other'

In [43]:
# снова взглянем на частоты категорий GEN_TITLE
out['GEN_TITLE'].value_counts(dropna=False)

Специалист                        7011
Рабочий                           3075
Не указано                        1366
Служащий                           904
Руководитель среднего звена        697
Работник сферы услуг               563
Высококвалифиц. специалист         549
Руководитель высшего звена         427
Индивидуальный предприниматель     217
Другое                             177
Руководитель низшего звена         136
Военнослужащий по контракту         88
Other                               13
Name: GEN_TITLE, dtype: int64

In [44]:
# теперь проверим работу класса RareGrouper2,
# категории с количеством менее 500 наблюдений запишем в
# отдельную категорию other, порог будет равен 500
raregroup2 = RareGrouper2(min_freq=500)
raregroup2.fit(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'] = raregroup2.transform(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'].value_counts(dropna=False)

Специалист    4886
Рабочий       2175
other         2005
Не указано     967
Служащий       623
Name: GEN_TITLE, dtype: int64

In [45]:
# создаем собственный класс Replacer, заменяющие отрицательные
# и нулевые значения на небольшие положительные (необходимо
# перед применением логарифмического и обратного преобразований)
class Replacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        repl: значение для замены
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X <= 0, self.repl_value, X)
        return X_replaced

In [46]:
# создаем собственный класс CorrLog, выполняюший 
# преобразование log(x/mean(x)+k)
class CorrLog(BaseEstimator, TransformerMixin):
    """
    Параметры:
    lower: нижнее пороговое значение
    a: константа
    k: корректировка асимметрии
    """
    def __init__(self, lower=0.001, k=0.2):
        self.lower = lower
        self.k = k
        self.mean = None
        
    def fit(self, X, y=None):
        self.mean = X.mean()
        return self
    
    def transform(self, X):
        X = np.log(X.clip(self.lower) / self.mean + self.k)
        return X

In [47]:
# создаем конвейер для количественных переменных DLQ_TO_PAYM, 
# LOAN_DLQ_NUM, PREVIOUS_CARD_NUM_UTILIZED, CHILD_TOTAL,
# LOAN_NUM_PAYM, LOAN_NUM_CLOSED, SHARE_INCOME, FST_SHARE,
# DEPENDANTS, AGE, OWN_AUTO, TERM, LOAN_MAX_DLQ, LOAN_NUM_OPEN,
# LOAN_MAX_DLQ_AMT, LOAN_NUM_TOTAL, PAYM_TO_LOAN, к которым
# не будут применены преобразования, максимизирующие нормальность
num_pipe = Pipeline([
    ('scaler', StandardScaler())
])

# создаем конвейер для количественных переменных WORK_TIME, 
# PERSONAL_INCOME, PAYMENT, CREDIT, LOAN_AVG_DLQ_AMT,
# к которым будет применено логарифмическое преобразование
num_log_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('repl', Replacer(repl_value=0.1)),
    ('log', FunctionTransformer(np.log, validate=False)),
    ('scaler', StandardScaler())
])

# создаем конвейер для количественной переменной FACT_LIVING_TERM,
# к которой применяем логарифмическое преобразование
# log(x/mean(x)+0.5)
fctlivingterm_corrlog0_5_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('corrlog0_5', CorrLog(k=0.5)),
    ('scaler', StandardScaler())
])

# создаем конвейер для количественной переменной FST_PAYMENT,
# к которой применяем логарифмическое преобразование
# log(x/mean(x)+0.1)
fstpayment_corrlog0_1_pipe = Pipeline([
    ('corrlog0_1', CorrLog(k=0.1)),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальной 
# переменной REGION_NM
regionnm_regroup_pipe = Pipeline([
    ('regroup', FunctionTransformer(collapse_categories, validate=False)),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем конвейер для категориальной 
# переменной GEN_TITLE
gentitle_delrare_pipe = Pipeline([
    ('raregroup', RareGrouper()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем конвейер для категориальной 
# переменной GEN_INDUSTRY
genindustry_delrare_pipe = Pipeline([
    ('raregroup2', RareGrouper2()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('num_log', num_log_pipe, num_columns_log),
                ('fctlivingterm_corrlog0_5', fctlivingterm_corrlog0_5_pipe, fctlivingterm_corrlog0_5),
                ('fstpayment_corrlog0_1', fstpayment_corrlog0_1_pipe, fstpayment_corrlog0_1),
                ('regionnm_regroup', regionnm_regroup_pipe, regionnm_regroup),
                ('gentitle_delrare', gentitle_delrare_pipe, gentitle_delrare),
                ('genindustry_delrare', genindustry_delrare_pipe, genindustry_delrare),
                ('cat', cat_pipe, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
pipe = Pipeline([('tf', transformer), 
                 ('logreg', LogisticRegression(solver='liblinear', penalty='l1', random_state=42))])

In [48]:
# задаем сетку гиперпараметров
param_grid = {'tf__gentitle_delrare__raregroup__threshold': [20, 300],
              'tf__genindustry_delrare__raregroup2__min_freq': [300, 500],
              'logreg__C': [0.14, 0.16, 0.18]}

In [49]:
# создаем экземпляр класса StratifiedKFold, 5-блочную
# стратифицированную перекрестную проверку
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [50]:
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs = GridSearchCV(pipe, 
                  param_grid, 
                  scoring='roc_auc',
                  cv=strat, 
                  return_train_score=False)

In [51]:
# выполняем поиск по сетке
gs.fit(train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение AUC
print('Наилучшее значение AUC: {:.3f}'.format(gs.best_score_))
# смотрим значение AUC на тестовой выборке
print('AUC на тестовом наборе: {:.3f}'.format(
    roc_auc_score(y_test, gs.predict_proba(test)[:, 1])))

Наилучшие значения гиперпараметров: {'logreg__C': 0.14, 'tf__genindustry_delrare__raregroup2__min_freq': 300, 'tf__gentitle_delrare__raregroup__threshold': 300}
Наилучшее значение AUC: 0.705
AUC на тестовом наборе: 0.692


In [52]:
# записываем CSV-файл в объект DataFrame 
fulldata = pd.read_csv('Data/Credit_OTP.csv', encoding='cp1251', sep=';')

In [53]:
# пишем функцию, выполняющую предварительную 
# обработку всех исторических данных
def preprocessing(df):
    # переменные, неверно записанные как количественные,
    # преобразуем в тип object
    object_columns = df.columns[df.columns.str.contains('FL|GENDER|OWN_AUTO|AGREEMENT')]
    df[object_columns] = df[object_columns].astype('object')

    # в указанных переменных заменяем запятую на точку в качестве
    # десятичного разделителя и преобразуем в тип float
    for column in ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 
                   'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']:
        df[column] = df[column].str.replace(',', '.').astype('float')
    
    # создаем список из переменных GEN_ INDUSTRY, GEN_TITLE, 
    # ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR
    work_cols = df.columns[df.columns.str.contains('GEN_I|GEN_T|JOB|ORG_')].tolist()

    # заменяем пропуски в переменных GEN_ INDUSTRY, GEN_TITLE, 
    # ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR  на "Не указано",
    # если в интересующей нас переменной есть пропуск 
    # И при этом переменная SOCSTATUS_PENS_FL имеет значение 1
    for col in work_cols:
        df[col] = np.where((df[col].isnull()) & (df['SOCSTATUS_PENS_FL'] == 1),
                           'Не указано', df[col])
    
    # заменяем пропуски в переменных GEN_ INDUSTRY, GEN_TITLE, 
    # ORG_TP_STATE, ORG_TP_FCAPITAL, JOB_DIR  модами,
    # если в интересующей нас переменной есть пропуск 
    # И при этом переменная SOCSTATUS_PENS_FL имеет значение 0
    for col in work_cols:
        df[col] = np.where((df[col].isnull()) & (df['SOCSTATUS_PENS_FL'] == 0),
                           df[col].value_counts().index[0], df[col])
    
    # заменяем пропуски в переменной REGION_NM модой
    df['REGION_NM'] = np.where(df['REGION_NM'].isnull(), 
                               df['REGION_NM'].value_counts().index[0], 
                               df['REGION_NM'])

    # пропуски в переменной TP_PROVINCE заменим значением
    # переменной FACT_ADDRESS_PROVINCE
    df['TP_PROVINCE'] = np.where(df['TP_PROVINCE'].isnull(), 
                                 df['FACT_ADDRESS_PROVINCE'], df['TP_PROVINCE'])
    
    # заменяем пропуски в переменной 
    # PREVIOUS_CARD_NUM_UTILIZED нулями
    df['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(df['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, 
                                                df['PREVIOUS_CARD_NUM_UTILIZED'])
    
    # удаляем идентификационную переменную AGREEMENT_RK,
    # потому что у нее количество уникальных значений
    # равно количеству наблюдений, переменную DL_DOCUMENT_FL, 
    # потому что у нее одно уникальное значение
    df.drop(['AGREEMENT_RK', 'DL_DOCUMENT_FL'], axis=1, inplace=True)
    
    # заменяем неверную категорию "ПОВОЛЖСКИЙ" на категорию "ПРИВОЛЖСКИЙ"
    df.loc[df['REGION_NM'] == 'ПОВОЛЖСКИЙ', 'REGION_NM'] = 'ПРИВОЛЖСКИЙ'
    
    # укрупняем категории переменной EDUCATION
    lst = ['Ученая степень', 'Два и более высших образования']
    df.loc[df['EDUCATION'].isin(lst), 'EDUCATION'] = 'Высшее'
    
    # укрупняем категории переменной OWN_AUTO
    df.loc[df['OWN_AUTO'] == '2', 'OWN_AUTO'] = '1'
    
    # создаем словарь со строковыми значениями для замены
    dct = {'Агинский Бурятский АО': 'Забайкальский край', 
           'Читинская область': 'Забайкальский край',
           'Усть-Ордынский Бурятский АО': 'Иркутская область',
           'Эвенкийский АО': 'Красноярский край'}

    # заменяем категории в переменных REG_ADDRESS_PROVINCE, POSTAL_ADDRESS_PROVINCE, 
    # FACT_ADDRESS_PROVINCE, TP_PROVINCE
    for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 
                'FACT_ADDRESS_PROVINCE', 'TP_PROVINCE']:
        df[col].replace(dct, inplace=True)
        
    # записываем редкие категории переменных REG_ADDRESS_PROVINCE, 
    # POSTAL_ADDRESS_PROVINCE, FACT_ADDRESS_PROVINCE в одну 
    # отдельную категорию
    for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE']:
        df[col] = np.where((df[col] == 'Хакасия')
                           | (df[col] == 'Ямало-Ненецкий АО')
                           | (df[col] == 'Магаданская область')
                           | (df[col] == 'Калмыкия')
                           | (df[col] == 'Дагестан')
                           | (df[col] == 'Чечня'), 
                           'ДРУГОЕ', df[col])

    # записываем редкие категории переменной TP_PROVINCE в одну отдельную категорию
    df['TP_PROVINCE'] = np.where((df['TP_PROVINCE'] == 'Сахалинская область')
                                 | (df['TP_PROVINCE'] == 'Еврейская АО')
                                 | (df['TP_PROVINCE'] == 'Магаданская область')
                                 | (df['TP_PROVINCE'] == 'Дагестан')
                                 | (df['TP_PROVINCE'] == 'Кабардино-Балкария'), 
                                 'ДРУГОЕ', df['TP_PROVINCE'])
    
    # записываем некоторые категории переменной JOB_DIR 
    # в отдельную категорию
    jobdir_lst = ['Реклама и маркетинг', 'Кадровая служба и секретариат',
                  'Пр-техн. обесп. и телеком.', 'Юридическая служба']
    df.loc[df['JOB_DIR'].isin(jobdir_lst), 'JOB_DIR'] = 'Другое'
    
    # создаем переменную FACT_TP_FL, которая принимает значение 1, если 
    # область фактического пребывания клиента и область торговой точки, 
    # где клиент брал последний кредит, совпадают, или 0
    # в противном случае
    df['FACT_TP_FL'] = np.where(df['FACT_ADDRESS_PROVINCE'] == df['TP_PROVINCE'], 
                                1, 0).astype('object')
    
    # значения переменной FACT_LIVING_TERM берем по модулю, 
    # чтобы избавиться от отрицательных значений
    df['FACT_LIVING_TERM'] = df['FACT_LIVING_TERM'].abs()
    
    a = 0.001
    # создаем переменную PAYMENT - сумму ежемесячного взноса
    # по кредиту, разделив сумму кредита (CREDIT)
    # на срок кредита (TERM)
    df['PAYMENT'] = (df['CREDIT'] + a) / (df['TERM'] + a)

    # создаем переменную SHARE_INCOME - долю дохода от суммы кредита
    df['SHARE_INCOME'] = (df['PERSONAL_INCOME'] + a) / (df['CREDIT'] + a)

    # вычитаем количество погашенных ссуд (LOAN_NUM_CLOSED)
    # из общего количество ссуд (LOAN_NUM_TOTAL)
    df['LOAN_NUM_OPEN'] = df['LOAN_NUM_TOTAL'] - df['LOAN_NUM_CLOSED']

    # создаем переменную PAYM_TO_LOAN – отношение количества 
    # платежей (LOAN_NUM_PAYM) к общему количеству ссуд (LOAN_NUM_TOTAL)
    df['PAYM_TO_LOAN'] = (df['LOAN_NUM_PAYM'] + a) / (df['LOAN_NUM_TOTAL'] + a)

    # создаем переменную DLQ_TO_PAYM – отношение количества просрочек 
    # (LOAN_DLQ_NUM) к общему количеству платежей (LOAN_NUM_PAYM)
    df['DLQ_TO_PAYM'] = (df['LOAN_DLQ_NUM'] + a) / (df['LOAN_NUM_PAYM'] + a)

    # создаем переменную FST_SHARE - отношение суммы первого платежа по кредиту
    # к объединенной сумме первого платежа (FST_PAYMENT) и кредита (CREDIT) 
    df['FST_SHARE'] = (df['FST_PAYMENT'] + a) / (df['FST_PAYMENT'] + df['CREDIT'])
    
    # на основе переменной PERSONAL_INCOME создаем временную переменную 
    # PERSONAL_INCOME_CAT, у которой метки категорий будут 
    # повторять метки категорий FAMILY_INCOME
    bins = [-np.inf, 5000, 10000, 20000, 50000, np.inf]
    lab = ['до 5000 руб.', 'от 5000 до 10000 руб.', 'от 10000 до 20000 руб.', 
           'от 20000 до 50000 руб.', 'свыше 50000 руб.']
    df['PERSONAL_INCOME_CAT'] = pd.cut(df['PERSONAL_INCOME'], bins, labels=lab)
    
    # создаем переменную PERSONAL_FAMILY_INCOME, которая принимает значение 1, если 
    # категория переменной PERSONAL_INCOME_CAT совпадает с категорией 
    # переменной FAMILY_INCOME, или 0 в противном случае, затем
    # преобразовываем в тип object
    df['PERSONAL_FAMILY_INCOME'] = np.where(df['PERSONAL_INCOME_CAT'] == df['FAMILY_INCOME'], 
                                            1, 0).astype('object')

    # удаляем переменную PERSONAL_INCOME_CAT
    df.drop('PERSONAL_INCOME_CAT', axis=1, inplace=True)
    
    # наблюдения, в которых количество лет проживания 
    # по месту фактического пребывания превышает
    # возраст, записываем как пропуски 
    df['FACT_LIVING_TERM'] = np.where(df['FACT_LIVING_TERM'] / 12 > df['AGE'],
                                      np.NaN, df['FACT_LIVING_TERM'])
    
    # наблюдения, в которых разница между возрастом и временем работы в годах 
    # меньше 16 (например, у 30-летнего время работы в годах составляет 20 лет,
    # получается, он работает с 10 лет), записываем как пропуски 
    df['WORK_TIME'] = np.where((df['AGE'] - df['WORK_TIME'] / 12) < 16, 
                                  np.NaN, df['WORK_TIME'])

In [54]:
# применяем функцию предварительной обработки 
# ко всем историческим данным
preprocessing(fulldata)

In [55]:
# создаем массив меток и массив признаков
y_fulldata = fulldata.pop('TARGET')

In [56]:
# записываем оптимальные значения гиперпараметров
best_params = gs.best_params_
# присваиваем итоговому конвейеру оптимальные 
# значения гиперпараметров
pipe.set_params(**best_params)
# обучаем итоговый конвейер с оптимальными значениями 
# гиперпараметров на всех исторических данных
pipe.fit(fulldata, y_fulldata)
# смотрим значение AUC
print('AUC на всей исторической выборке: {:.3f}'.format(
    roc_auc_score(y_fulldata, pipe.predict_proba(fulldata)[:, 1])))

AUC на всей исторической выборке: 0.722


In [57]:
# записываем CSV-файл, содержащий новые данные,
# в объект DataFrame
newdata = pd.read_csv('Data/Credit_OTP_new.csv', encoding='cp1251', sep=';')

In [58]:
# выполняем предварительную обработку
# новых данных
preprocessing(newdata)
y_new = newdata.pop('TARGET').values

In [59]:
# смотрим AUC на новых данных
print("AUC на выборке новых данных: {:.3f}".
      format(roc_auc_score(y_new, pipe.predict_proba(newdata)[:, 1])))

AUC на выборке новых данных: 0.708
